In [1]:
import sys

sys.path.append('/root/lib/')

import ROOT as r
import os
import json
import pandas as pd
import uproot 
import awkward as ak
import array as arr
import numpy as np
import sys

sys.path.append(os.getcwd() + '/../utilities/')
from milliqanProcessor import *
from milliqanScheduler import *
from milliqanCuts import *
from milliqanPlotter import *

%jsroot on
%load_ext autoreload
%autoreload 2

Welcome to JupyROOT 6.30/02


In [2]:
#get list of files to look at
files = []

dataDir = '/store/user/milliqan/trees/v34/1400/'
for filename in os.listdir(dataDir):
    if not filename.endswith('root'): continue
    if not int(filename.split('Run')[1].split('.')[0]) >= 1410: continue
    files.append(dataDir+filename)

files

['/store/user/milliqan/trees/v34/1400/MilliQan_Run1410.1_v34.root',
 '/store/user/milliqan/trees/v34/1400/MilliQan_Run1411.100_v34.root',
 '/store/user/milliqan/trees/v34/1400/MilliQan_Run1411.101_v34.root',
 '/store/user/milliqan/trees/v34/1400/MilliQan_Run1411.102_v34.root',
 '/store/user/milliqan/trees/v34/1400/MilliQan_Run1411.103_v34.root',
 '/store/user/milliqan/trees/v34/1400/MilliQan_Run1411.104_v34.root',
 '/store/user/milliqan/trees/v34/1400/MilliQan_Run1411.105_v34.root',
 '/store/user/milliqan/trees/v34/1400/MilliQan_Run1411.106_v34.root',
 '/store/user/milliqan/trees/v34/1400/MilliQan_Run1411.107_v34.root',
 '/store/user/milliqan/trees/v34/1400/MilliQan_Run1411.108_v34.root',
 '/store/user/milliqan/trees/v34/1400/MilliQan_Run1411.109_v34.root',
 '/store/user/milliqan/trees/v34/1400/MilliQan_Run1411.10_v34.root',
 '/store/user/milliqan/trees/v34/1400/MilliQan_Run1411.110_v34.root',
 '/store/user/milliqan/trees/v34/1400/MilliQan_Run1411.111_v34.root',
 '/store/user/milliqan/

In [22]:
@mqCut
def countTriggers(self, cutName=None, trigNum=2):
    triggers = ak.firsts(self.events['tTrigger'])
    binary_trig = 1 << (trigNum-1)
    #print("Binary", trigNum, binary_trig)
    thisTrig = ak.count(triggers[triggers == binary_trig], axis=None)
    #print("instances of trigger {}, {}".format(trigNum, thisTrig))
    self.events[cutName] = thisTrig

print(countTriggers.__name__)

countTriggers


In [25]:
@mqCut
def firstEvent(self):
    events = ak.firsts(self.events['event'])
    mask = np.zeros(len(events), dtype=bool)
    mask[0] = True
    mask = ak.Array(mask)
    self.events['firsts'] = mask

In [30]:
#define a file list to run over
filelist = files

print(filelist)

#define the necessary branches to run over
branches = ['event', 'tTrigger', 'boardsMatched', 'pickupFlag']

#define the milliqan cuts object
mycuts = milliqanCuts()

#require pulses are not pickup
pickupCut = mycuts.getCut(mycuts.pickupCut, 'pickupCut', cut=True, branches=branches)

#require that all digitizer boards are matched
boardMatchCut = mycuts.getCut(mycuts.boardsMatched, 'boardMatchCut', cut=True, branches=branches)

setattr(milliqanCuts, 'countTriggers', countTriggers)
setattr(milliqanCuts, 'firstEvent', firstEvent)

trig1 = mycuts.getCut(mycuts.countTriggers, "trig1", trigNum=1)
trig2 = mycuts.getCut(mycuts.countTriggers, "trig2", trigNum=2)
trig3 = mycuts.getCut(mycuts.countTriggers, "trig3", trigNum=3)
trig4 = mycuts.getCut(mycuts.countTriggers, "trig4", trigNum=4)
trig5 = mycuts.getCut(mycuts.countTriggers, "trig5", trigNum=5)
trig7 = mycuts.getCut(mycuts.countTriggers, "trig7", trigNum=7)
trig9 = mycuts.getCut(mycuts.countTriggers, "trig9", trigNum=9)
trig10 = mycuts.getCut(mycuts.countTriggers, "trig10", trigNum=10)
trig11 = mycuts.getCut(mycuts.countTriggers, "trig11", trigNum=11)
trig13 = mycuts.getCut(mycuts.countTriggers, "trig13", trigNum=13)

#define milliqan plotter
myplotter = milliqanPlotter()

#create root histogram 
bins = 200
xmin = 0
xmax = 400

h_triggers = r.TH1F("h_triggers", "Triggers", bins, xmin, xmax)
h_trig1 = r.TH1F("h_trig1", "Trigger 1", bins, xmin, xmax)
h_trig2 = r.TH1F("h_trig2", "Trigger 2", bins, xmin, xmax)
h_trig3 = r.TH1F("h_trig3", "Trigger 3", bins, xmin, xmax)
h_trig4 = r.TH1F("h_trig4", "Trigger 4", bins, xmin, xmax)
h_trig5 = r.TH1F("h_trig5", "Trigger 5", bins, xmin, xmax)
h_trig7 = r.TH1F("h_trig7", "Trigger 7", bins, xmin, xmax)
h_trig9 = r.TH1F("h_trig9", "Trigger 9", bins, xmin, xmax)
h_trig10 = r.TH1F("h_trig10", "Trigger 10", bins, xmin, xmax)
h_trig11 = r.TH1F("h_trig11", "Trigger 11", bins, xmin, xmax)
h_trig13 = r.TH1F("h_trig13", "Trigger 13", bins, xmin, xmax)


#add root histogram to plotter
myplotter.addHistograms(h_triggers, 'tTrigger')
myplotter.addHistograms(h_trig1, 'trig1', cut='firsts')
myplotter.addHistograms(h_trig2, 'trig2', cut='firsts')
myplotter.addHistograms(h_trig3, 'trig3', cut='firsts')
myplotter.addHistograms(h_trig4, 'trig4', cut='firsts')
myplotter.addHistograms(h_trig5, 'trig5', cut='firsts')
myplotter.addHistograms(h_trig7, 'trig7', cut='firsts')
myplotter.addHistograms(h_trig9, 'trig9', cut='firsts')
myplotter.addHistograms(h_trig10, 'trig10', cut='firsts')
myplotter.addHistograms(h_trig11, 'trig11', cut='firsts')
myplotter.addHistograms(h_trig13, 'trig13', cut='firsts')

#defining the cutflow
#cutflow = [boardMatchCut, pickupCut, 
cutflow = [trig1] 

'''trig2, trig3, trig4, trig5, trig7, trig9, trig10, trig11, trig13,
            mycuts.firstEvent, myplotter.dict['h_triggers'], 
            myplotter.dict['h_trig1'], myplotter.dict['h_trig2'], myplotter.dict['h_trig3'], myplotter.dict['h_trig4'], 
            myplotter.dict['h_trig5'], myplotter.dict['h_trig7'], myplotter.dict['h_trig9'], myplotter.dict['h_trig10'], 
            myplotter.dict['h_trig11'], myplotter.dict['h_trig13'], mycuts.getCutflowCounts
            ]'''

#create a schedule of the cuts
myschedule = milliQanScheduler(cutflow, mycuts, myplotter)

#print out the schedule
myschedule.printSchedule()

#create the milliqan processor object
myiterator = milliqanProcessor(filelist, branches, myschedule, step_size=1000, max_events=2000)

#run the milliqan processor
myiterator.run()

['/store/user/milliqan/trees/v34/1400/MilliQan_Run1410.1_v34.root', '/store/user/milliqan/trees/v34/1400/MilliQan_Run1411.100_v34.root', '/store/user/milliqan/trees/v34/1400/MilliQan_Run1411.101_v34.root', '/store/user/milliqan/trees/v34/1400/MilliQan_Run1411.102_v34.root', '/store/user/milliqan/trees/v34/1400/MilliQan_Run1411.103_v34.root', '/store/user/milliqan/trees/v34/1400/MilliQan_Run1411.104_v34.root', '/store/user/milliqan/trees/v34/1400/MilliQan_Run1411.105_v34.root', '/store/user/milliqan/trees/v34/1400/MilliQan_Run1411.106_v34.root', '/store/user/milliqan/trees/v34/1400/MilliQan_Run1411.107_v34.root', '/store/user/milliqan/trees/v34/1400/MilliQan_Run1411.108_v34.root', '/store/user/milliqan/trees/v34/1400/MilliQan_Run1411.109_v34.root', '/store/user/milliqan/trees/v34/1400/MilliQan_Run1411.10_v34.root', '/store/user/milliqan/trees/v34/1400/MilliQan_Run1411.110_v34.root', '/store/user/milliqan/trees/v34/1400/MilliQan_Run1411.111_v34.root', '/store/user/milliqan/trees/v34/1400

Warning in <TROOT::Append>: Replacing existing TH1: h_triggers (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h_trig1 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h_trig2 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h_trig3 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h_trig4 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h_trig5 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h_trig7 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h_trig9 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h_trig10 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h_trig11 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h_trig13 (Potential memory leak).


In [62]:
c1 = r.TCanvas("c1", "c1", 800, 600)

Warning in <TCanvas::Constructor>: Deleting canvas with same name: c1


In [63]:
h_triggers.Draw()
c1.Draw()

NameError: name 'h_triggers' is not defined

In [7]:
fin = r.TFile.Open("triggerRateStudy.root", 'read')
fin.ls()
h_trig1 = fin.Get("h_trig1")
h_trig2 = fin.Get("h_trig2")
h_trig3 = fin.Get("h_trig3")
h_trig4 = fin.Get("h_trig4")
h_trig5 = fin.Get("h_trig5")
h_trig7 = fin.Get("h_trig7")
h_trig9 = fin.Get("h_trig9")
h_trig10 = fin.Get("h_trig10")
h_trig11 = fin.Get("h_trig11")
h_trig13 = fin.Get("h_trig13")

TFile**		triggerRateStudy.root	
 TFile*		triggerRateStudy.root	
  KEY: TH1F	h_trig1;1	Trigger 1
  KEY: TH1F	h_trig2;1	Trigger 2
  KEY: TH1F	h_trig3;1	Trigger 3
  KEY: TH1F	h_trig4;1	Trigger 4
  KEY: TH1F	h_trig5;1	Trigger 5
  KEY: TH1F	h_trig7;1	Trigger 7
  KEY: TH1F	h_trig9;1	Trigger 9
  KEY: TH1F	h_trig10;1	Trigger 10
  KEY: TH1F	h_trig11;1	Trigger 11
  KEY: TH1F	h_trig13;1	Trigger 13


In [65]:
parameters = {}

h_trig1.Draw()
f1 = r.TF1("f1", "gaus", 1, 100)
h_trig1.Fit("f1", "0")
mean = f1.GetParameter(1)
sigma = f1.GetParameter(2)
parameters['trig1'] = [round(mean, 2), round(sigma, 2)]
c1.Draw()

****************************************
Minimizer is Minuit2 / Migrad
Chi2                      =      5829.32
NDf                       =           26
Edm                       =   4.9922e-07
NCalls                    =          189
Constant                  =      944.435   +/-   14.9096     
Mean                      =      25.5815   +/-   0.0689414   
Sigma                     =      5.25712   +/-   0.0493882    	 (limited)


In [66]:
h_trig2.Draw()
c1.Draw()
f1 = r.TF1("f1", "gaus", 100, 300)
h_trig2.Fit("f1", "0")
mean = f1.GetParameter(1)
sigma = f1.GetParameter(2)
parameters['trig2'] = [round(mean, 2), round(sigma, 2)]


****************************************
Minimizer is Minuit2 / Migrad
Chi2                      =      5934.79
NDf                       =          110
Edm                       =  3.85713e-07
NCalls                    =          338
Constant                  =      308.334   +/-   4.74165     
Mean                      =      197.582   +/-   0.205344    
Sigma                     =      15.7683   +/-   0.13514      	 (limited)


In [67]:
h_trig3.Draw()
c1.Draw()
f1 = r.TF1("f1", "gaus", 1, 100)
h_trig3.Fit("f1", "0")
mean = f1.GetParameter(1)
sigma = f1.GetParameter(2)
parameters['trig3'] = [round(mean, 2), round(sigma, 2)]


****************************************
Minimizer is Minuit2 / Migrad
Chi2                      =      5765.83
NDf                       =           41
Edm                       =  4.19829e-07
NCalls                    =          157
Constant                  =      714.251   +/-   11.3506     
Mean                      =       43.393   +/-   0.0891676   
Sigma                     =      6.96635   +/-   0.0669125    	 (limited)


In [68]:
h_trig4.Draw()
c1.Draw()
f1 = r.TF1("f1", "gaus", 1, 100)
h_trig4.Fit("f1", "0")
mean = f1.GetParameter(1)
sigma = f1.GetParameter(2)
parameters['trig4'] = [round(mean, 2), round(sigma, 2)]


****************************************
Minimizer is Minuit2 / Migrad
Chi2                      =      5830.28
NDf                       =           39
Edm                       =   3.3618e-06
NCalls                    =          155
Constant                  =      649.739   +/-   10.3846     
Mean                      =      54.2069   +/-   0.097244    
Sigma                     =      7.62677   +/-   0.074017     	 (limited)


In [70]:
h_trig5.Draw()
c1.Draw()
f1 = r.TF1("f1", "gaus", 1, 50)
h_trig5.Fit("f1", "0")
mean = f1.GetParameter(1)
sigma = f1.GetParameter(2)
parameters['trig5'] = [round(mean, 2), round(sigma, 2)]


****************************************
Minimizer is Minuit2 / Migrad
Chi2                      =      5392.92
NDf                       =            8
Edm                       =  4.21027e-07
NCalls                    =          106
Constant                  =      1654.57   +/-   29.7389     
Mean                      =      7.44592   +/-   0.0451723   
Sigma                     =      3.15725   +/-   0.0439567    	 (limited)


In [73]:
h_trig7.Draw()
c1.Draw()
f1 = r.TF1("f1", "gaus", 50, 150)
h_trig7.Fit("f1", "0", "0", 50, 150)
mean = f1.GetParameter(1)
sigma = f1.GetParameter(2)
parameters['trig7'] = [round(mean, 2), round(sigma, 2)]


****************************************
Minimizer is Minuit2 / Migrad
Chi2                      =      95.4176
NDf                       =           46
Edm                       =  2.14642e-07
NCalls                    =           65
Constant                  =      475.087   +/-   7.73199     
Mean                      =      95.8192   +/-   0.134422    
Sigma                     =      10.3548   +/-   0.104919     	 (limited)


In [74]:
h_trig9.Draw()
c1.Draw()
f1 = r.TF1("f1", "gaus", 1, 150)
h_trig9.Fit("f1", "0")
mean = f1.GetParameter(1)
sigma = f1.GetParameter(2)
parameters['trig9'] = [round(mean, 2), round(sigma, 2)]


****************************************
Minimizer is Minuit2 / Migrad
Chi2                      =      5954.53
NDf                       =           43
Edm                       =  9.38458e-08
NCalls                    =          138
Constant                  =      501.305   +/-   8.22741     
Mean                      =      59.2239   +/-   0.145382    
Sigma                     =      9.52768   +/-   0.0963239    	 (limited)


In [75]:
h_trig10.Draw()
c1.Draw()
f1 = r.TF1("f1", "gaus", 100, 350)
h_trig10.Fit("f1", "0", "0", 100, 350)
mean = f1.GetParameter(1)
sigma = f1.GetParameter(2)
parameters['trig10'] = [round(mean, 2), round(sigma, 2)]


****************************************
Minimizer is Minuit2 / Migrad
Chi2                      =      1245.86
NDf                       =           89
Edm                       =  1.66517e-08
NCalls                    =          138
Constant                  =      272.853   +/-   4.95678     
Mean                      =      241.959   +/-   0.212409    
Sigma                     =      14.5743   +/-   0.166292     	 (limited)


In [76]:
h_trig11.Draw()
c1.Draw()
f1 = r.TF1("f1", "gaus", 1, 100)
h_trig11.Fit("f1", "0", "0", 1, 50)
mean = f1.GetParameter(1)
sigma = f1.GetParameter(2)
parameters['trig11'] = [round(mean, 2), round(sigma, 2)]


****************************************
Minimizer is Minuit2 / Migrad
Chi2                      =      5858.04
NDf                       =           17
Edm                       =   4.0712e-08
NCalls                    =          126
Constant                  =      875.859   +/-   13.435      
Mean                      =      20.8802   +/-   0.0722774   
Sigma                     =      5.64496   +/-   0.0487205    	 (limited)


In [77]:
h_trig13.Draw()
c1.Draw()
f1 = r.TF1("f1", "gaus", 100, 250)
h_trig13.Fit("f1","0", "0", 120, 220)
mean = f1.GetParameter(1)
sigma = f1.GetParameter(2)
parameters['trig13'] = [round(mean, 2), round(sigma, 2)]


****************************************
Minimizer is Minuit2 / Migrad
Chi2                      =      85.3741
NDf                       =           45
Edm                       =  5.72578e-06
NCalls                    =           57
Constant                  =      375.079   +/-   5.94074     
Mean                      =      169.494   +/-   0.169549    
Sigma                     =      12.7611   +/-   0.117421     	 (limited)


In [158]:
'''fout = r.TFile.Open("triggerRateStudy.root", 'recreate')

fout.cd()
h_trig1.Write()
h_trig2.Write()
h_trig3.Write()
h_trig4.Write()
h_trig5.Write()
h_trig7.Write()
h_trig9.Write()
h_trig10.Write()
h_trig11.Write()
h_trig13.Write()
fout.Close()'''

In [78]:
h_trig2.SetLineColor(2)
h_trig3.SetLineColor(3)
h_trig4.SetLineColor(4)
h_trig5.SetLineColor(6)
h_trig7.SetLineColor(7)
h_trig9.SetLineColor(46)
h_trig10.SetLineColor(51)
h_trig11.SetLineColor(23)
h_trig13.SetLineColor(94)

h_trig1.SetTitle("Triggers Fired per File (Runs 1400+)")
h_trig1.GetXaxis().SetTitle("Triggers per File")
h_trig1.GetYaxis().SetTitle("Number of Files")

h_trig1.Draw()
h_trig2.Draw("same")
h_trig3.Draw("same")
h_trig4.Draw("same")
h_trig5.Draw("same")
h_trig7.Draw("same")
h_trig9.Draw("same")
h_trig10.Draw("same")
h_trig11.Draw("same")
h_trig13.Draw("same")

l1 = r.TLegend(0.6, 0.9, 0.6, 0.9)
l1.AddEntry(h_trig1, "Trigger 1: \mu {} \pm {}".format(parameters['trig1'][0], parameters['trig1'][1]), 'l')
l1.AddEntry(h_trig2, "Trigger 2: \mu {} \pm {}".format(parameters['trig2'][0], parameters['trig2'][1]), 'l')
l1.AddEntry(h_trig3, "Trigger 3: \mu {} \pm {}".format(parameters['trig3'][0], parameters['trig3'][1]), 'l')
l1.AddEntry(h_trig4, "Trigger 4: \mu {} \pm {}".format(parameters['trig4'][0], parameters['trig4'][1]), 'l')
l1.AddEntry(h_trig5, "Trigger 5: \mu {} \pm {}".format(parameters['trig5'][0], parameters['trig5'][1]), 'l')
l1.AddEntry(h_trig7, "Trigger 7: \mu {} \pm {}".format(parameters['trig7'][0], parameters['trig7'][1]), 'l')
l1.AddEntry(h_trig9, "Trigger 9: \mu {} \pm {}".format(parameters['trig9'][0], parameters['trig9'][1]), 'l')
l1.AddEntry(h_trig10, "Trigger 10: \mu {} \pm {}".format(parameters['trig10'][0], parameters['trig10'][1]), 'l')
l1.AddEntry(h_trig11, "Trigger 11: \mu {} \pm {}".format(parameters['trig11'][0], parameters['trig11'][1]), 'l')
l1.AddEntry(h_trig13, "Trigger 13: \mu {} \pm {}".format(parameters['trig13'][0], parameters['trig13'][1]), 'l')


l1.Draw()
c1.Draw()



Warning in <TLegend::Paint>: Legend too large to be automatically placed; a default position is used
